In [30]:
import numpy as np
import pandas as pd

In [31]:
df = pd.read_csv("train_data.csv")
raw_data = pd.read_csv("train_data.csv")
df_test = pd.read_csv("test_data.csv")

In [32]:
df.head()

,client_id,cato_code,gender,country,citizenship,age_days,client_days,opsos_code,phone_digits,max_dup,...,salary_10,transfers_amount_in_8,transfers_count_in_8,transfers_amount_out_8,transfers_count_out_8,transactions_amount_8,transactions_count_8,auth_count_8,salary_8,label
0,2152686,17.0,1.0,0.0,0.0,11643.0,130.0,9.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1712777,2.0,NaN,0.0,0.0,18095.0,994.0,5.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,458915,NaN,0.0,0.0,0.0,7955.0,1144.0,7.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,0.037251,0.033580,0.03358,NaN,0
3,530220,NaN,0.0,0.0,0.0,22580.0,847.0,0.0,6.0,1.0,...,0.03656,NaN,NaN,NaN,NaN,0.115086,0.033581,0.03358,0.092161,0
4,1103654,12.0,0.0,0.0,0.0,13107.0,547.0,12.0,6.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [33]:
# finding features with all NAN values
drop_features = list(df.loc[:,((df.isnull().sum()/df.shape[0])==1).values].columns)
drop_features

['transactions_amount_9',
 'transactions_count_9',
 'transactions_amount_10',
 'transactions_count_10',
 'transfers_amount_in_8',
 'transfers_count_in_8',
 'transfers_amount_out_8',
 'transfers_count_out_8']

In [34]:
df.columns

Index(['client_id', 'cato_code', 'gender', 'country', 'citizenship',
       'age_days', 'client_days', 'opsos_code', 'phone_digits', 'max_dup',
       'monthly_installment', 'date_close', 'deposit_sum_KZT',
       'deposit_sum_USD', 'deposit_sum_EUR', 'deal_sum_kzt_express',
       'expired_day_express', 'deal_start_date_express',
       'plan_end_date_express', 'deal_sum_kzt_product', 'expired_day_product',
       'deal_start_date_product', 'plan_end_date_product',
       'transfers_amount_in_9', 'transfers_count_in_9',
       'transfers_amount_out_9', 'transfers_count_out_9',
       'transactions_amount_9', 'transactions_count_9', 'auth_count_9',
       'salary_9', 'transfers_amount_in_10', 'transfers_count_in_10',
       'transfers_amount_out_10', 'transfers_count_out_10',
       'transactions_amount_10', 'transactions_count_10', 'auth_count_10',
       'salary_10', 'transfers_amount_in_8', 'transfers_count_in_8',
       'transfers_amount_out_8', 'transfers_count_out_8',
       'tra

# Feature engineering

In [35]:
def drop_cols(df):
    drop_features = list(df.loc[:,((df.isnull().sum()/df.shape[0])==1).values].columns)
    df.drop(drop_features, axis=1, inplace=True)
    return df

In [36]:
def fill_cols(df):
    fill_features_0 = ['deposit_sum_KZT', 'deposit_sum_USD', 'deposit_sum_EUR']
    df[fill_features_0] = df[fill_features_0].fillna(0, axis=1)
    feature_fill = ['transfers_amount_in_9', 'transfers_count_in_9', 'transfers_amount_out_9', 'transfers_count_out_9', 'transfers_amount_in_10', 'transfers_count_in_10', 'transfers_amount_out_10', 'transfers_count_out_10', 'transactions_amount_8', 'transactions_count_8', 'auth_count_9', 'auth_count_10', 'auth_count_8']
    for feature in feature_fill:
        df[feature].fillna(df[feature].mean(), inplace=True)
    return df

In [37]:
def mean_salary(df):
    salary_list = ['salary_8', 'salary_9', 'salary_10']
    df[salary_list] = df[salary_list].fillna(0, axis=1)
    df['mean_salary'] = df[salary_list].sum(axis=1)
    df['mean_salary'] = df['mean_salary'] / 3
    df.drop(salary_list, axis=1, inplace=True)
    
    extreme_low = 0
    low = df[df.mean_salary>0]['mean_salary'].describe()[4]
    medium = df[df.mean_salary>0]['mean_salary'].describe()[5]
    high = df[df.mean_salary>0]['mean_salary'].describe()[6]
    extreme_high = df[df.mean_salary>0]['mean_salary'].describe()[7]

    salaries = [0, 1, 2, 3, 4]
    df.loc[df['mean_salary'] == extreme_low, 'salary_groups'] = salaries[0]
    df.loc[df['mean_salary'] > extreme_low, 'salary_groups'] = salaries[1]
    df.loc[df['mean_salary'] > low, 'salary_groups'] = salaries[2]
    df.loc[df['mean_salary'] > medium, 'salary_groups'] = salaries[3]
    df.loc[df['mean_salary'] > high, 'salary_groups'] = salaries[4]
    
    return df

In [38]:
def deposit(df):
    # Filling deposit columns with 0
    fill_features_0 = ['deposit_sum_KZT', 'deposit_sum_USD', 'deposit_sum_EUR']
    df[fill_features_0] = df[fill_features_0].fillna(0, axis=1)
    # Adding 3 new columns for each group
    df['no_deposit'] = 0
    df['closed_deposit'] = 0
    df['not_closed_deposit'] = 0
    # Filling 
    df.loc[df[fill_features_0].sum(axis=1) > 0 , 'not_closed_deposit'] = 1
    df.loc[pd.notnull(df['date_close']), 'closed_deposit'] = 1
    df.loc[((df[fill_features_0].sum(axis=1) == 0)  & (pd.isnull(df['date_close']))), 'no_deposit'] = 1
    #df.loc[pd.isnull(df['date_close']), 'no_deposit'] = 1
    # Dropping initial deposit columns
    df.drop(fill_features_0, axis=1, inplace=True)
    return df

In [39]:
def desired_added(df):
    not_closed_deposit_index = df.loc[df['not_closed_deposit'] == 1, 'client_id'].tolist()
    salary_X_index = df.loc[df['mean_salary'] > 0, 'client_id'].tolist()
    shared_user = list(set(not_closed_deposit_index).intersection(salary_X_index))
    df['desired_user'] = 0
    df.loc[df['client_id'].isin(shared_user), 'desired_user'] = 1
    return df

In [40]:
def date_to_num(col):
    col = pd.to_datetime(col)
    c = list()
    for row in col:
        if pd.isnull(row.year):
            c.append(0)
        elif row.year < 2021:
            c.append(1)
        elif row.year >= 2021 and row.year <= 2023:
            c.append(2)
        else:
            c.append(3)
    return c

In [41]:
import datetime 

def dates(df):
    df['dsde'] = date_to_num(df['deal_start_date_express'])
    df['pede'] = date_to_num(df['plan_end_date_express'])
    df['pedp'] = date_to_num(df['plan_end_date_product'])
    df['dsdp'] = date_to_num(df['deal_start_date_product'])
    df['dc'] = date_to_num(df['date_close'])
    
    date = ['deal_start_date_express', 'plan_end_date_express', 'plan_end_date_product', 'deal_start_date_product', 'date_close']
    for time in date:
        df[time] = pd.to_datetime(df[time])
        df[time] = df[time].map(datetime.datetime.toordinal)
    return df

In [42]:
df = raw_data.copy()
df = drop_cols(df)
df = fill_cols(df)
df = mean_salary(df)
df = deposit(df)
df = desired_added(df)
df = dates(df)

In [43]:
df.head()

,client_id,cato_code,gender,country,citizenship,age_days,client_days,opsos_code,phone_digits,max_dup,...,salary_groups,no_deposit,closed_deposit,not_closed_deposit,desired_user,dsde,pede,pedp,dsdp,dc
0,2152686,17.0,1.0,0.0,0.0,11643.0,130.0,9.0,5.0,1.0,...,0.0,1,0,0,0,0,0,0,0,0
1,1712777,2.0,NaN,0.0,0.0,18095.0,994.0,5.0,5.0,1.0,...,0.0,1,0,0,0,0,0,0,0,0
2,458915,NaN,0.0,0.0,0.0,7955.0,1144.0,7.0,5.0,1.0,...,0.0,0,1,0,0,0,0,0,0,2
3,530220,NaN,0.0,0.0,0.0,22580.0,847.0,0.0,6.0,1.0,...,3.0,1,0,0,0,0,0,0,0,0
4,1103654,12.0,0.0,0.0,0.0,13107.0,547.0,12.0,6.0,1.0,...,0.0,1,0,0,0,0,0,0,0,0


In [44]:
df[df.mean_salary>0].boxplot(['mean_salary'])

<AxesSubplot:>

In [45]:
df.corr().label.sort_values(ascending=False)

label                      1.000000
pedp                       0.253926
dsdp                       0.239066
plan_end_date_product      0.239066
deal_start_date_product    0.239040
auth_count_10              0.149568
auth_count_9               0.134895
transfers_count_out_10     0.098091
auth_count_8               0.093687
deal_sum_kzt_product       0.087642
expired_day_product        0.085945
transfers_count_out_9      0.080508
transactions_count_8       0.076190
transfers_count_in_10      0.053779
dsde                       0.047748
transfers_count_in_9       0.044070
deal_start_date_express    0.042635
plan_end_date_express      0.042615
desired_user               0.040845
pede                       0.040512
mean_salary                0.031165
not_closed_deposit         0.022793
salary_groups              0.022730
expired_day_express        0.015317
max_dup                    0.014001
transactions_amount_8      0.011291
transfers_amount_in_10     0.011235
transfers_amount_out_10    0

In [46]:
features = abs(df.corr().label.sort_values(ascending=False))[1:9].index

In [47]:
X_test = df_test.copy()
X_test = drop_cols(X_test)
X_test = fill_cols(X_test)
X_test = mean_salary(X_test)
X_test = deposit(X_test)
X_test = desired_added(X_test)
X_test = dates(X_test)

In [48]:
'''from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[features], df.label, random_state=31, train_size=0.8)'''
X_train = df[features]
y_train = df.label
X_test = X_test[features]

In [49]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [21]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train_scaled, y_train.values.ravel()) 

# Predict the Test set results
y_pred = model.predict(X_test_scaled)

C:\Users\user\miniconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
C:\Users\user\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:00:52] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [50]:
from catboost import CatBoostClassifier

cat_features = list(range(0, X_train_scaled.shape[1]))

clf = CatBoostClassifier()
clf.fit(X_train_scaled, y_train.values.ravel())

Learning rate set to 0.191761
0:	learn: 0.3104337	total: 193ms	remaining: 3m 12s
1:	learn: 0.1741229	total: 314ms	remaining: 2m 36s
2:	learn: 0.1050309	total: 448ms	remaining: 2m 28s
3:	learn: 0.0748623	total: 575ms	remaining: 2m 23s
4:	learn: 0.0613272	total: 738ms	remaining: 2m 26s
5:	learn: 0.0544974	total: 860ms	remaining: 2m 22s
6:	learn: 0.0510645	total: 983ms	remaining: 2m 19s
7:	learn: 0.0484475	total: 1.12s	remaining: 2m 18s
8:	learn: 0.0471337	total: 1.25s	remaining: 2m 18s
9:	learn: 0.0459969	total: 1.4s	remaining: 2m 18s
10:	learn: 0.0452197	total: 1.54s	remaining: 2m 18s
11:	learn: 0.0448520	total: 1.67s	remaining: 2m 17s
12:	learn: 0.0445866	total: 1.8s	remaining: 2m 16s
13:	learn: 0.0442547	total: 1.94s	remaining: 2m 16s
14:	learn: 0.0439661	total: 2.08s	remaining: 2m 16s
15:	learn: 0.0438418	total: 2.2s	remaining: 2m 15s
16:	learn: 0.0436489	total: 2.35s	remaining: 2m 16s
17:	learn: 0.0434281	total: 2.48s	remaining: 2m 15s
18:	learn: 0.0432969	total: 2.61s	remaining: 2m

159:	learn: 0.0402490	total: 22.4s	remaining: 1m 57s
160:	learn: 0.0402405	total: 22.5s	remaining: 1m 57s
161:	learn: 0.0402310	total: 22.6s	remaining: 1m 56s
162:	learn: 0.0402192	total: 22.7s	remaining: 1m 56s
163:	learn: 0.0402146	total: 22.9s	remaining: 1m 56s
164:	learn: 0.0401998	total: 23s	remaining: 1m 56s
165:	learn: 0.0401823	total: 23.1s	remaining: 1m 56s
166:	learn: 0.0401772	total: 23.3s	remaining: 1m 56s
167:	learn: 0.0401699	total: 23.4s	remaining: 1m 55s
168:	learn: 0.0401602	total: 23.5s	remaining: 1m 55s
169:	learn: 0.0401579	total: 23.6s	remaining: 1m 55s
170:	learn: 0.0401496	total: 23.8s	remaining: 1m 55s
171:	learn: 0.0401421	total: 23.9s	remaining: 1m 55s
172:	learn: 0.0401367	total: 24s	remaining: 1m 54s
173:	learn: 0.0401322	total: 24.1s	remaining: 1m 54s
174:	learn: 0.0401276	total: 24.3s	remaining: 1m 54s
175:	learn: 0.0401196	total: 24.4s	remaining: 1m 54s
176:	learn: 0.0401137	total: 24.5s	remaining: 1m 53s
177:	learn: 0.0401058	total: 24.6s	remaining: 1m 5

316:	learn: 0.0391467	total: 44.4s	remaining: 1m 35s
317:	learn: 0.0391420	total: 44.6s	remaining: 1m 35s
318:	learn: 0.0391344	total: 44.7s	remaining: 1m 35s
319:	learn: 0.0391255	total: 44.8s	remaining: 1m 35s
320:	learn: 0.0391209	total: 45s	remaining: 1m 35s
321:	learn: 0.0391155	total: 45.1s	remaining: 1m 34s
322:	learn: 0.0391086	total: 45.2s	remaining: 1m 34s
323:	learn: 0.0391027	total: 45.3s	remaining: 1m 34s
324:	learn: 0.0390913	total: 45.5s	remaining: 1m 34s
325:	learn: 0.0390856	total: 45.6s	remaining: 1m 34s
326:	learn: 0.0390847	total: 45.7s	remaining: 1m 34s
327:	learn: 0.0390802	total: 45.9s	remaining: 1m 33s
328:	learn: 0.0390707	total: 46s	remaining: 1m 33s
329:	learn: 0.0390655	total: 46.1s	remaining: 1m 33s
330:	learn: 0.0390577	total: 46.3s	remaining: 1m 33s
331:	learn: 0.0390522	total: 46.4s	remaining: 1m 33s
332:	learn: 0.0390456	total: 46.6s	remaining: 1m 33s
333:	learn: 0.0390401	total: 46.8s	remaining: 1m 33s
334:	learn: 0.0390323	total: 46.9s	remaining: 1m 3

472:	learn: 0.0383532	total: 1m 6s	remaining: 1m 13s
473:	learn: 0.0383489	total: 1m 6s	remaining: 1m 13s
474:	learn: 0.0383452	total: 1m 6s	remaining: 1m 13s
475:	learn: 0.0383476	total: 1m 6s	remaining: 1m 13s
476:	learn: 0.0383377	total: 1m 6s	remaining: 1m 13s
477:	learn: 0.0383331	total: 1m 6s	remaining: 1m 12s
478:	learn: 0.0383322	total: 1m 6s	remaining: 1m 12s
479:	learn: 0.0383281	total: 1m 7s	remaining: 1m 12s
480:	learn: 0.0383186	total: 1m 7s	remaining: 1m 12s
481:	learn: 0.0383149	total: 1m 7s	remaining: 1m 12s
482:	learn: 0.0383088	total: 1m 7s	remaining: 1m 12s
483:	learn: 0.0383035	total: 1m 7s	remaining: 1m 11s
484:	learn: 0.0382954	total: 1m 7s	remaining: 1m 11s
485:	learn: 0.0382908	total: 1m 7s	remaining: 1m 11s
486:	learn: 0.0382847	total: 1m 7s	remaining: 1m 11s
487:	learn: 0.0382786	total: 1m 8s	remaining: 1m 11s
488:	learn: 0.0382753	total: 1m 8s	remaining: 1m 11s
489:	learn: 0.0382701	total: 1m 8s	remaining: 1m 11s
490:	learn: 0.0382642	total: 1m 8s	remaining: 

629:	learn: 0.0376306	total: 1m 26s	remaining: 50.9s
630:	learn: 0.0376249	total: 1m 26s	remaining: 50.7s
631:	learn: 0.0376208	total: 1m 26s	remaining: 50.6s
632:	learn: 0.0376198	total: 1m 27s	remaining: 50.4s
633:	learn: 0.0376196	total: 1m 27s	remaining: 50.3s
634:	learn: 0.0376180	total: 1m 27s	remaining: 50.2s
635:	learn: 0.0376150	total: 1m 27s	remaining: 50s
636:	learn: 0.0376127	total: 1m 27s	remaining: 49.9s
637:	learn: 0.0376057	total: 1m 27s	remaining: 49.7s
638:	learn: 0.0376013	total: 1m 27s	remaining: 49.6s
639:	learn: 0.0376010	total: 1m 27s	remaining: 49.4s
640:	learn: 0.0375976	total: 1m 28s	remaining: 49.3s
641:	learn: 0.0375948	total: 1m 28s	remaining: 49.1s
642:	learn: 0.0375919	total: 1m 28s	remaining: 49s
643:	learn: 0.0375905	total: 1m 28s	remaining: 48.9s
644:	learn: 0.0375885	total: 1m 28s	remaining: 48.7s
645:	learn: 0.0375874	total: 1m 28s	remaining: 48.6s
646:	learn: 0.0375866	total: 1m 28s	remaining: 48.4s
647:	learn: 0.0375850	total: 1m 28s	remaining: 48.

785:	learn: 0.0369707	total: 1m 47s	remaining: 29.3s
786:	learn: 0.0369678	total: 1m 47s	remaining: 29.1s
787:	learn: 0.0369667	total: 1m 47s	remaining: 29s
788:	learn: 0.0369651	total: 1m 47s	remaining: 28.8s
789:	learn: 0.0369637	total: 1m 47s	remaining: 28.7s
790:	learn: 0.0369582	total: 1m 48s	remaining: 28.6s
791:	learn: 0.0369574	total: 1m 48s	remaining: 28.4s
792:	learn: 0.0369546	total: 1m 48s	remaining: 28.3s
793:	learn: 0.0369506	total: 1m 48s	remaining: 28.2s
794:	learn: 0.0369453	total: 1m 48s	remaining: 28s
795:	learn: 0.0369437	total: 1m 48s	remaining: 27.9s
796:	learn: 0.0369359	total: 1m 48s	remaining: 27.7s
797:	learn: 0.0369310	total: 1m 49s	remaining: 27.6s
798:	learn: 0.0369255	total: 1m 49s	remaining: 27.5s
799:	learn: 0.0369206	total: 1m 49s	remaining: 27.3s
800:	learn: 0.0369171	total: 1m 49s	remaining: 27.2s
801:	learn: 0.0369137	total: 1m 49s	remaining: 27.1s
802:	learn: 0.0369100	total: 1m 49s	remaining: 26.9s
803:	learn: 0.0369062	total: 1m 49s	remaining: 26.

942:	learn: 0.0364264	total: 2m 8s	remaining: 7.79s
943:	learn: 0.0364256	total: 2m 9s	remaining: 7.66s
944:	learn: 0.0364224	total: 2m 9s	remaining: 7.52s
945:	learn: 0.0364191	total: 2m 9s	remaining: 7.38s
946:	learn: 0.0364172	total: 2m 9s	remaining: 7.25s
947:	learn: 0.0364150	total: 2m 9s	remaining: 7.11s
948:	learn: 0.0364120	total: 2m 9s	remaining: 6.97s
949:	learn: 0.0364118	total: 2m 9s	remaining: 6.84s
950:	learn: 0.0364086	total: 2m 10s	remaining: 6.7s
951:	learn: 0.0364022	total: 2m 10s	remaining: 6.57s
952:	learn: 0.0364004	total: 2m 10s	remaining: 6.43s
953:	learn: 0.0363958	total: 2m 10s	remaining: 6.29s
954:	learn: 0.0363917	total: 2m 10s	remaining: 6.16s
955:	learn: 0.0363896	total: 2m 10s	remaining: 6.02s
956:	learn: 0.0363840	total: 2m 10s	remaining: 5.88s
957:	learn: 0.0363795	total: 2m 11s	remaining: 5.75s
958:	learn: 0.0363764	total: 2m 11s	remaining: 5.61s
959:	learn: 0.0363740	total: 2m 11s	remaining: 5.47s
960:	learn: 0.0363701	total: 2m 11s	remaining: 5.34s
96

In [59]:
preds = clf.predict(X_test_scaled)

In [60]:
preds.shape

(463739,)

In [61]:
result = pd.DataFrame({'id': df_test.client_id, 'target': preds})
result.to_csv('submission.csv', index=False)